<a id="0"></a>
# <center style="background-color:#63809e; color:white;">HackerEarth-Deep-Learning-Challenge-Snakes-in-the-hood!</center>

<center><img src="https://tblogqus.com/wp-content/uploads/2020/11/HackerEarth-Deep-Learning-Challenge-Snakes-in-the-hood.png" width=70%></center>

## Problem statement

The government has been facing a long-standing issue of wild animals entering residential areas due to various reasons. It's of critical importance that if any such dangerous animal is encountered, the concerned authority should be notified immediately. Reptiles, especially snakes, are among the most dangerous animals and they often enter residential areas. 

Recently due to an incident of a youngster getting bitten by a snake, the government decided to install cameras at every corner of the road to detect snakes and other animals.

You have been hired as a Deep Learning engineer to create a sophisticated model that can detect the breed of a snake from its image.

## Data description

This data set consists of the following two columns:

Column Name	Description
* image_id :	Name of the image file
* breed    :	Snake breed [35 different breeds]

The data folder consists of two folders and two .csv files. The details are as follows:

* train: Contains 5508 images for 35 classes 
* test: Contains 2361 images
* train.csv: 5508 x 2
* test.csv: 2361 x 1

<a id="0"></a>
## <center style="background-color:yellow; color:black;">If you are facing in some import issues of fastai.callbacks or hooks please copy and edit this notebook to resolve the issue.</center>

### <center style="background-color:orange; color:black;">Please upvote this notebook if it helps you to implement the problem. Soon , I will update with a diffreent approach using Tranfer learning.</center>

I was facing the same import issues in fastAi. Thanks @rajat ranjan for this kernel.

## Importing Necessary Libraries

In [ ]:
import numpy as np 
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import os
import glob
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from pathlib import Path
from fastai import *
from fastai.vision import *
import torch
from fastai.callbacks.hooks import *
from tqdm import tqdm_notebook

# import required libraries 
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16,ResNet101,ResNet101V2
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

from sklearn.preprocessing import LabelEncoder

In [ ]:
data_folder = Path("../input/hackerearth-deep-learning-identify-the-snake-breed/dataset")
data_path = "../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train"
path = os.path.join(data_path , "*jpg")
path

In [ ]:
files = glob.glob(path)
data=[]
name_img_mapper = {}
for file in tqdm_notebook(files):
    fn = file.split('/')[-1]
    image = cv2.imread(file)
    data.append(file)
    name_img_mapper[fn] = image

In [ ]:
## read the csv data files
train_df = pd.read_csv('../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train.csv')
test_df = pd.read_csv('../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/test.csv')

In [ ]:
# Concatining the .jpg extension for the sake of simplicity
train = train_df.copy()
test = test_df.copy()

train['image_id'] = train['image_id']+str('.jpg')
test['image_id'] = test['image_id']+str('.jpg')

print(train.head())
print(test.head())

In [ ]:
train.groupby('breed').count()

In [ ]:
sns.countplot(x='breed' , data=train)

In [ ]:
train_images = data

In [ ]:
#using Label encoder to encode the training labes into numeric
le =LabelEncoder()
training_labels=le.fit_transform(train['breed'])

#train_df.loc[train_df['breed']== 'pantherophis-spiloides']['image_id'][:3].tolist()

In [ ]:
training_labels

In [ ]:
#Showing Sample Images
train.loc[train['breed']== 'pantherophis-spiloides']['image_id'][:3].tolist()

In [ ]:
def plot_class(cat):
    
    fetch = train.loc[train_df['breed']== cat][:3]
    images_names = train.loc[train_df['breed']== cat]['image_id'][:3].tolist()
#     print(images_names)
    fig = plt.figure(figsize=(20,15))
    
    for i, img_name in enumerate(images_names):
        plt.subplot(1,3 ,i+1)
        plt.imshow(name_img_mapper[img_name])
    plt.show()

In [ ]:
plt.imshow(name_img_mapper['299c932a68.jpg'])

In [ ]:
plot_class('pantherophis-spiloides')

In [ ]:
##transformations to be done to images
tfms = get_transforms(do_flip=True,flip_vert=False ,max_rotate=20.0, max_zoom=1.3, max_lighting=0.5, max_warp=0.1, p_affine=0.2,
                      p_lighting=0.55)
#, xtra_tfms=zoom_crop(scale=(0.9,1.8), do_rand=True, p=0.8))

## create databunch of test set to be passed
test_img = ImageList.from_df(test, path=data_folder, folder='test')

In [ ]:
np.random.seed(145)
## create source of train image databunch
src = (ImageList.from_df(train, path=data_folder, folder='train')
       .split_by_rand_pct(0.10)
       #.split_none()
       .label_from_df()
       .add_test(test_img))

In [ ]:
data = (src.transform(tfms, size=224,padding_mode='reflection',resize_method=ResizeMethod.SQUISH)
        .databunch(path='.', bs=64, device= torch.device('cuda:0')).normalize(imagenet_stats))

In [ ]:
data.show_batch(rows=3, figsize=(12,12))

In [ ]:
print(data.classes)

In [ ]:
learn = cnn_learner(data=data, base_arch=models.resnet101, metrics=[FBeta(beta=1, average='macro'), accuracy],
                    callback_fns=ShowGraph)

In [ ]:
#lets find the correct learning rate to be used from lr finder
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
lr = 1e-03
learn.fit_one_cycle(10, slice(lr))

In [ ]:
#lets plot the lr finder record
learn.unfreeze()
learn.lr_find()

learn.recorder.plot(suggestion=True)

In [ ]:
learn.fit_one_cycle(10,slice(1e-06,lr/10))

In [ ]:
#lets see the most mis-classified images (on validation set)
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(9, figsize=(15,10))

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12),dpi=60)

In [ ]:
##learn.TTA improves score further. lets see for the validation set
pred_val,y = learn.TTA(ds_type=DatasetType.Valid)
from sklearn.metrics import f1_score, accuracy_score
valid_preds = [np.argmax(pred_val[i])+1 for i in range(len(pred_val))]
valid_preds = np.array(valid_preds)
y = np.array(y+1)
accuracy_score(valid_preds,y),f1_score(valid_preds,y, average='weighted')

In [ ]:
preds,_ = learn.TTA(ds_type=DatasetType.Test)
#preds,_ = learn.get_preds(ds_type = DatasetType.Test)
labelled_preds = [np.argmax(preds[i])+1 for i in range(len(preds))]

labelled_preds = np.array(labelled_preds)

In [ ]:
df = pd.DataFrame({'image_id':test_df['image_id'], 'breed':labelled_preds})
df

In [ ]:
category = {'agkistrodon-contortrix':1,
            'agkistrodon-piscivorus':2, 
            'coluber-constrictor':3, 
            'crotalus-atrox':4, 
            'crotalus-horridus':5, 
            'crotalus-ruber':6, 
            'crotalus-scutulatus':7, 
            'crotalus-viridis':8, 
            'diadophis-punctatus':9, 
            'haldea-striatula':10, 
            'heterodon-platirhinos':11, 
            'lampropeltis-californiae':12, 
            'lampropeltis-triangulum':13, 
            'masticophis-flagellum':14, 
            'natrix-natrix':15, 
            'nerodia-erythrogaster':16, 
            'nerodia-fasciata':17, 
            'nerodia-rhombifer':18, 
            'nerodia-sipedon':19, 
            'opheodrys-aestivus':20, 
            'pantherophis-alleghaniensis':21, 
            'pantherophis-emoryi':22, 
            'pantherophis-guttatus':23, 
            'pantherophis-obsoletus':24, 
            'pantherophis-spiloides':25, 
            'pantherophis-vulpinus':26, 
            'pituophis-catenifer':27, 
            'rhinocheilus-lecontei':28, 
            'storeria-dekayi':29, 
            'storeria-occipitomaculata':30, 
            'thamnophis-elegans':31, 
            'thamnophis-marcianus':32, 
            'thamnophis-proximus':33, 
            'thamnophis-radix':34, 
            'thamnophis-sirtalis':35}


rev_category = {val: key for key, val in category.items()}
df['breed'] = df['breed'].map(rev_category)
df.to_csv('submission_test.csv', index=False)

In [ ]:
df.head()

In [ ]:
from IPython.display import HTML
def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

create_download_link(filename = 'submission.csv')

------------------------------------

In [ ]:
!pip install -U efficientnet

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
# import required libraries 
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

from sklearn.preprocessing import LabelEncoder

In [ ]:
# Load train and test csv file for image class
train_original = pd.read_csv('../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train.csv')
test_original = pd.read_csv('../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/test.csv')

In [ ]:
train = train_original.copy()
test = test_original.copy()

train['image_id'] = train['image_id']+str('.jpg')
test['image_id'] = test['image_id']+str('.jpg')

print(train.head())
print(test.head())

In [ ]:
#using Label encoder to encode the training labes into numeric
le =LabelEncoder()
train['breed'] = le.fit_transform(train['breed'])

In [ ]:
train_img=[]
train_label=[]
j=0
path='../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/train'
for i in tqdm(train['image_id']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(224,224))
    img=img.astype('float32')
    train_img.append(img)
    train_label.append(train['breed'][j])
    j=j+1

In [ ]:
test_img=[]
path='../input/hackerearth-deep-learning-identify-the-snake-breed/dataset/test'
for i in tqdm(test['image_id']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(224,224))
    img=img.astype('float32')
    test_img.append(img)

In [ ]:
#from keras import applications
from efficientnet.model import EfficientNetB7
from keras import callbacks
from keras.models import Sequential

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagenerator = ImageDataGenerator(
        rescale=1/255,
        validation_split=0.10,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest') 


datagenerator.fit(train_img)

In [ ]:
train_img=np.array(train_img)
test_img=np.array(test_img)
train_label=np.array(train_label)
print(train_img.shape)
print(test_img.shape)
print(train_label.shape)

In [ ]:
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
from keras.models import Sequential

base_model = EfficientNetB7(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')

base_model.trainable = False

model = Sequential([
  base_model, 
  Dense(1024, activation='relu'),
  Dropout(0.5),
  Dense(256, activation='relu'),
  Dense(35, activation='softmax'),
])